Construct Entry | EC number | Sequence csv for erxprot as for CLEAN's datasets

In [1]:
from src.utils import load_json
from collections import defaultdict
import pandas as pd
import subprocess

In [45]:
# Create csv index in format of other datasets

f = load_json('../data/erxprot/sequences/sequences.json')
id2ec = defaultdict(list)
id2seq = {}
for ec, v in f.items():
    for elt in v:
        id = list(elt.keys())[0]
        seq = elt[id]
        id2ec[id].append(ec)
        id2seq[id] = seq

print(len(id2ec), len(id2seq))

data = []
for k in id2ec.keys():
    data.append([k, ';'.join(id2ec[k]), id2seq[k]])

header = ['Entry', 'EC number', 'Sequence']
df = pd.DataFrame(data, columns=header)
df.to_csv('../data/erxprot/erxprot.csv', index=False, sep='\t')

104200 104200


In [15]:
# # Re-name erxprot pt files ONLY RUN ONCE

# for dir in ['../data/erxprot/clean/', '../data/erxprot/esm/']:
#     for fn in os.listdir(dir):
#         id = fn.split('|')[0].strip()
#         cmd = ["mv", dir + fn, dir + id + '.pt']
#         subprocess.run(cmd)

In [50]:
# Overlap in enzymes

erx_csv = pd.read_csv('../data/erxprot/erxprot.csv', delimiter='\t')
erx_csv.set_index('Entry', inplace=True)

swissprot = pd.read_csv('../data/swissprot/swissprot.csv', delimiter='\t')
swissprot.set_index('Entry', inplace=True)

print(len(set(erx_csv.index) & set(swissprot.index)) / len(set(erx_csv.index) | set(swissprot.index)))
idx_intersect = list(set(erx_csv.index) & set(swissprot.index))

0.14423657628370382


In [3]:
# Overlap in EC numbers

erx_csv = pd.read_csv('../data/erxprot/erxprot.csv', delimiter='\t')
erx_csv.set_index('Entry', inplace=True)

swissprot = pd.read_csv('../data/swissprot/swissprot.csv', delimiter='\t')
swissprot.set_index('Entry', inplace=True)

erx_ec = []
for elt in erx_csv.loc[:, 'EC number']:
    for sub in elt.split(';'):
        erx_ec.append(sub)

swiss_ec = []
for elt in swissprot.loc[:, 'EC number']:
    for sub in elt.split(';'):
        swiss_ec.append(sub)

print(len(set(erx_ec) & set(swiss_ec)) / len(set(erx_ec) | set(swiss_ec)))
idx_intersect = list(set(erx_ec) & set(swiss_ec))

0.8378031154649108


In [12]:
# Overlap in EC numbers

first = 'swissprot'
second = 'erxprot'

first_df = pd.read_csv(f"../data/{first}/{first}.csv", delimiter='\t')
first_df.set_index('Entry', inplace=True)

second_df = pd.read_csv(f"../data/{second}/{second}.csv", delimiter='\t')
second_df.set_index('Entry', inplace=True)

first_ec = []
for elt in first_df.loc[:, 'EC number']:
    first_ec.append(elt.split(';')[0])

second_ec = []
for elt in second_df.loc[:, 'EC number']:
    second_ec.append(elt.split(';')[0])

print(len(set(first_ec) & set(second_ec)) / len(set(first_ec) | set(second_ec)))
print(len(set(first_ec) & set(second_ec)) / len(set(second_ec)))
ec_intersect = list(set(first_ec) & set(second_ec))

0.8055647020085611
0.8140076526368325


In [13]:
idx_intersect

['2.1.4.2',
 '1.14.12.14',
 '3.5.1.116',
 '1.2.1.100',
 '3.4.17.18',
 '1.14.18.2',
 '1.1.1.270',
 '7.2.2.21',
 '1.2.1.4',
 '2.7.1.19',
 '3.4.23.23',
 '3.1.4.39',
 '2.7.1.148',
 '3.4.14.2',
 '1.1.1.17',
 '3.2.1.141',
 '2.1.1.273',
 '3.1.1.95',
 '3.1.3.87',
 '1.14.99.15',
 '2.6.1.59',
 '2.6.1.117',
 '3.5.1.15',
 '4.1.1.96',
 '1.14.14.82',
 '3.4.21.6',
 '1.14.14.79',
 '1.13.11.12',
 '2.5.1.88',
 '1.1.5.11',
 '1.6.5.9',
 '2.8.2.14',
 '3.2.2.1',
 '2.1.1.22',
 '2.1.1.348',
 '4.2.3.31',
 '4.1.2.48',
 '4.2.3.175',
 '1.3.1.24',
 '3.5.1.10',
 '3.2.1.166',
 '1.4.1.16',
 '2.4.1.360',
 '3.2.1.80',
 '2.4.1.44',
 '2.8.2.1',
 '7.6.2.11',
 '1.13.11.52',
 '1.97.1.9',
 '2.3.1.40',
 '1.2.1.38',
 '3.1.21.5',
 '1.17.2.1',
 '1.1.1.236',
 '3.5.1.26',
 '4.1.1.76',
 '1.14.99.59',
 '2.1.1.308',
 '1.11.2.1',
 '2.5.1.46',
 '1.2.1.27',
 '2.4.2.62',
 '4.1.99.3',
 '1.1.1.289',
 '1.3.3.13',
 '3.4.24.59',
 '2.1.1.199',
 '1.5.3.19',
 '2.1.1.261',
 '3.4.24.44',
 '3.5.3.9',
 '3.1.6.4',
 '1.14.15.3',
 '1.14.14.39',
 '7.5.2

In [55]:
# Get consistent / discrepant ecs, update erxport
# w/ swissprot ec + extras
cons = []
discrep = []

for idx in idx_intersect:
    swiss = swissprot.loc[idx, 'EC number']
    erx = erx_csv.loc[idx, 'EC number']
    if sorted(swiss.split(';')) == sorted(erx.split(';')):
        cons.append(idx)
    else:
        discrep.append(idx)
        erx_extra = list(set(erx.split(';')) - set(swiss.split(';')))
        updated = swiss.split(';') + erx_extra
        erx_csv.loc[idx, 'EC number'] = ';'.join(updated)


erx_csv.to_csv('../data/erxprot/erxprot.csv', sep='\t')

In [53]:
len(cons), len(discrep), len(cons) / len(idx_intersect), len(discrep) / len(idx_intersect)

(39912, 1883, 0.9549467639669817, 0.04505323603301831)